In [15]:
import sys
sys.path.append('D:\GR1\Code')
from vnstock3 import Vnstock
import pandas as pd
import time
from backend.database.pgsql import PostgresManager
from datetime import datetime, timedelta

postgre = PostgresManager("GR1_data")
stock = Vnstock().stock(symbol='TCB',source='VCI')
current_time = datetime.now().strftime("%Y-%m-%d")

invalid escape sequence '\G'
invalid escape sequence '\G'
invalid escape sequence '\G'
2024-12-09 14:58:53 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [7]:
df = stock.trading.price_board(['VCB'])
# df.columns = df.columns.droplevel()
# df.columns = [col.strip() for col in df.columns]
# Bây giờ df.columns sẽ là các tên cột đơn giản
print(df.to_string())

  listing                                                                                                                                                                                 match                                                                                                                                                                                                                               bid_ask                                                                                                                                          
   symbol ceiling  floor ref_price stock_type exchange last_trading_date listed_share   type       id                                          organ_name prior_close_price benefit match_price match_vol accumulated_volume accumulated_value avg_match_price highest lowest match_type foreign_sell_volume foreign_buy_volume current_room  total_room total_accumulated_value total_accumulated_volume reference_price bid_1_price bid_1_volume bid_2

In [4]:
df = stock.quote.history(start='2024-12-5', end=current_time, symbol= 'AAA', interval ='1m')
print(df.to_string())

                   time  open  high   low  close  volume
0   2024-12-05 09:15:00  8.80  8.80  8.77   8.77   20600
1   2024-12-05 09:16:00  8.77  8.77  8.77   8.77   15800
2   2024-12-05 09:17:00  8.77  8.77  8.75   8.75    1100
3   2024-12-05 09:18:00  8.75  8.75  8.75   8.75     200
4   2024-12-05 09:20:00  8.75  8.75  8.74   8.74   12200
5   2024-12-05 09:21:00  8.74  8.74  8.74   8.74    2600
6   2024-12-05 09:23:00  8.74  8.75  8.72   8.75    2000
7   2024-12-05 09:26:00  8.75  8.75  8.74   8.75     100
8   2024-12-05 09:27:00  8.75  8.75  8.75   8.75    2300
9   2024-12-05 09:28:00  8.75  8.75  8.75   8.75    1000
10  2024-12-05 09:29:00  8.75  8.75  8.75   8.75     100
11  2024-12-05 09:30:00  8.75  8.75  8.75   8.75   15400
12  2024-12-05 09:31:00  8.75  8.75  8.75   8.75     500
13  2024-12-05 09:33:00  8.75  8.75  8.75   8.75     400
14  2024-12-05 09:35:00  8.75  8.75  8.73   8.73    2100
15  2024-12-05 09:37:00  8.73  8.74  8.73   8.74    1000
16  2024-12-05 09:40:00  8.74  

In [ ]:
symbols = stock.listing.symbols_by_exchange()

# Convert the data to a pandas DataFrame
df = pd.DataFrame(symbols)
df = df[~df['exchange'].isin(['DELISTED', 'BOND'])]


postgre.create_table_and_insert_dataframe(df, "danh_sach_ma_ck")

Inserted 1679 rows into table danh_sach_ma_ck.
Table 'danh_sach_ma_ck' created successfully.


In [ ]:
#Tạo bảng mã CK cho từng sàn

groups = ["HOSE", "HNX", "UPCOM"]

# Vòng lặp để xử lý cho từng sàn
for group in groups:
    # Lấy danh sách mã chứng khoán cho từng sàn
    symbols = stock.listing.symbols_by_group(group)
    
    # Chuyển đổi dữ liệu thành DataFrame
    df = pd.DataFrame(symbols)
    
    # Lọc các mã có chỉ số chia hết cho 4
    df_filtered = df.iloc[df.index % 4 == 0]
    
    # Tạo bảng và chèn dữ liệu vào PostgreSQL
    table_name = f"ma_ck_{group.lower()}"
    postgre.insert_dataframe_to_table(df=df_filtered, table_name=table_name)

Inserted 104 rows into table ma_ck_hose.
Inserted 79 rows into table ma_ck_hnx.
Inserted 221 rows into table ma_ck_upcom.


In [ ]:
symbols = ["VN30F1M", "VN30F2M"]
df = pd.DataFrame(symbols, columns=["symbol"])
postgre.create_table_and_insert_dataframe(df, "hop_dong_tuong_lai")

Table 'hop_dong_tuong_lai' was already existed
Inserted 1200 rows into table cp_day.
Inserted data from 0 - VN30F1M successfully.
Inserted 1213 rows into table cp_day.
Inserted data from 1 - VN30F2M successfully.


In [ ]:
symbols = ["VNINDEX", "HNXINDEX","UPCOMINDEX", "VN30", "HNX30"]
df = pd.DataFrame(symbols, columns=["symbol"])
postgre.create_table_and_insert_dataframe(df, "ma_index")

Table 'ma_index' created successfully.
Inserted 5 rows into table ma_index.
Inserted 1215 rows into table cp_day.
Inserted data from 0 - VNINDEX successfully.
Inserted 1215 rows into table cp_day.
Inserted data from 1 - HNXINDEX successfully.
Inserted 1215 rows into table cp_day.
Inserted data from 2 - UPCOMINDEX successfully.
Inserted 1215 rows into table cp_day.
Inserted data from 3 - VN30 successfully.
Inserted 1215 rows into table cp_day.
Inserted data from 4 - HNX30 successfully.


In [7]:

df = stock.listing.symbols_by_group('VN30')
df = pd.DataFrame(df, columns=["symbol"])
postgre.insert_dataframe_to_table(df, 'ma_vn30')
df = stock.listing.symbols_by_group('HNX30')
df = pd.DataFrame(df, columns=["symbol"])
postgre.insert_dataframe_to_table(df, 'ma_hnx30')

haha
Table 'ma_vn30' created successfully.
Inserted 30 rows into table ma_vn30.
haha
Table 'ma_hnx30' created successfully.
Inserted 30 rows into table ma_hnx30.


In [ ]:
# Data CP với interval = "1D" của từng mã
symbols = postgre.query_table(table_name="ma_ck_niemyet_all")
symbols = symbols["symbol"].tolist()

MAX_RETRIES = 3       # Số lần thử lại tối đa
DELAY_BETWEEN_REQUESTS = 3  # Thời gian chờ giữa các yêu cầu (giây)
DELAY_ON_429 = 6      # Thời gian chờ khi gặp lỗi 429 (giây)

for index, symbol in enumerate(symbols):
    retry_count = 0
    while retry_count < MAX_RETRIES:
        try:
            # Lấy dữ liệu cổ phiếu
            df = stock.quote.history(start='2024-11-20', end=current_time, symbol= symbol)
            df['symbol'] = symbol
            
            # Thêm dữ liệu vào PostgreSQL
            postgre.insert_dataframe_to_table(df, 'cp_day')
            print(f"Inserted data from {index} - {symbol} successfully.")
            
            # Thời gian chờ giữa các yêu cầu
            time.sleep(DELAY_BETWEEN_REQUESTS)
            break  # Thoát khỏi vòng lặp retry nếu thành công

        except Exception as e:
            if "429" in str(e):  # Kiểm tra lỗi 429
                print(f"Error 429: Too many requests for {index} - {symbol}. Retrying after {DELAY_ON_429} seconds.")
                time.sleep(DELAY_ON_429)  # Chờ lâu hơn khi gặp lỗi 429
                retry_count += 1
            else:
                print(f"Error with {index} - {symbol}: {e}")
                break  # Thoát vòng lặp retry nếu lỗi khác 429

Inserted 3 rows into table cp_day.
Inserted data from 0 - ABC successfully.
Inserted 1 rows into table cp_day.
Inserted data from 1 - NSG successfully.
Inserted 3 rows into table cp_day.
Inserted data from 2 - EID successfully.
Inserted 3 rows into table cp_day.
Inserted data from 3 - TVT successfully.
Inserted 1 rows into table cp_day.
Inserted data from 4 - NUE successfully.
Inserted 3 rows into table cp_day.
Inserted data from 5 - SAC successfully.
Inserted 1 rows into table cp_day.
Inserted data from 6 - PRO successfully.
Inserted 1 rows into table cp_day.
Inserted data from 7 - PPE successfully.
Inserted 3 rows into table cp_day.
Inserted data from 8 - PV2 successfully.
Inserted 3 rows into table cp_day.
Inserted data from 9 - SD9 successfully.
Inserted 3 rows into table cp_day.
Inserted data from 10 - IVS successfully.
Inserted 1 rows into table cp_day.
Inserted data from 11 - IME successfully.
Error with 12 - BED: Không tìm thấy dữ liệu. Vui lòng kiểm tra lại mã chứng khoán hoặc

In [10]:
# Cập nhật giá cp theo thời gian thực "1m"
current_time = datetime.now().strftime("%Y-%m-%d")
symbols = postgre.query_table(table_name="ma_ck_niemyet_all")
symbols = symbols["symbol"].tolist()

for index, symbol in enumerate(symbols):
    try:
        # Lấy latest_updated từ database
        latest_updated = postgre.get_latest_updated_timestamp(symbol, 'cp_min')
        
        # Lấy dữ liệu từ API
        df = stock.quote.history(
            start=latest_updated.strftime("%Y-%m-%d"), 
            end=current_time, 
            symbol=symbol, 
            interval='1m'
        )
        
        # Lọc dữ liệu chỉ lấy những bản ghi sau latest_updated
        df_filtered = df[df['time'] >= latest_updated].copy()
        
        # Kiểm tra nếu có dữ liệu mới
        if not df_filtered.empty:
            # Thêm cột symbol vào DataFrame
            df_filtered.loc[:, 'symbol'] = symbol
            
            # Chèn dữ liệu mới vào database
            postgre.insert_dataframe_to_table(df_filtered, 'cp_min')
            
            print(f"{index + 1}/{len(symbols)} - Thành công: Đã cập nhật {len(df_filtered)} dòng cho {symbol} {latest_updated}")
        else:
            print(f"{index + 1}/{len(symbols)} - Không có dữ liệu mới cho {symbol}")
    
    except Exception as e:
        print(f"{index + 1}/{len(symbols)} - Lỗi: Không thể cập nhật dữ liệu cho {symbol} - {e}")

Inserted 1 rows into table cp_min.
1/409 - Thành công: Đã cập nhật 1 dòng cho ABC 2024-11-13 14:55:00
Inserted 1 rows into table cp_min.
2/409 - Thành công: Đã cập nhật 1 dòng cho NSG 2024-11-08 13:00:00
Inserted 1 rows into table cp_min.
3/409 - Thành công: Đã cập nhật 1 dòng cho EID 2024-11-13 13:07:00
Inserted 1 rows into table cp_min.
4/409 - Thành công: Đã cập nhật 1 dòng cho TVT 2024-11-13 14:12:00
Inserted 1 rows into table cp_min.
5/409 - Thành công: Đã cập nhật 1 dòng cho NUE 2024-11-11 13:28:00
Inserted 1 rows into table cp_min.
6/409 - Thành công: Đã cập nhật 1 dòng cho SAC 2024-11-13 14:59:00
Inserted 1 rows into table cp_min.
7/409 - Thành công: Đã cập nhật 1 dòng cho PRO 2024-09-17 11:29:00
Inserted 1 rows into table cp_min.
8/409 - Thành công: Đã cập nhật 1 dòng cho PPE 2024-11-01 13:15:00
Inserted 1 rows into table cp_min.
9/409 - Thành công: Đã cập nhật 1 dòng cho PV2 2024-11-13 14:45:00
Inserted 1 rows into table cp_min.
10/409 - Thành công: Đã cập nhật 1 dòng cho SD9

KeyboardInterrupt: 

In [3]:
# Đặt tên bảng mới
table_name = 'stock_price_board'

# Truy vấn danh sách symbols từ ba bảng khác nhau
symbols_hnx = postgre.query_table(table_name='ma_ck_hnx', columns='symbol')
symbols_hose = postgre.query_table(table_name='ma_ck_hose', columns='symbol')
symbols_upcom = postgre.query_table(table_name='ma_ck_upcom', columns='symbol')

# Kết hợp danh sách symbols
all_symbols = pd.concat([symbols_hnx, symbols_hose, symbols_upcom]).drop_duplicates().reset_index(drop=True)
symbols = all_symbols['symbol'].to_list()

# Poll the price board data for each symbol
price_data = stock.trading.price_board(symbols_list=symbols)
price_data.columns = price_data.columns.droplevel(0)
required_columns = [
    'symbol', 'ceiling', 'floor', 'ref_price', 'stock_type', 
    'exchange', 'last_trading_date', 'listed_share', 'type', 
    'id', 'organ_name', 'prior_close_price', 'match_price', 
    'match_vol', 'accumulated_volume', 'accumulated_value', 
    'avg_match_price', 'highest', 'lowest', 'match_type', 
    'foreign_sell_volume', 'foreign_buy_volume', 'current_room', 
    'total_room', 'bid_1_price', 'bid_1_volume', 'bid_2_price', 
    'bid_2_volume', 'bid_3_price', 'bid_3_volume', 'ask_1_price', 
    'ask_1_volume', 'ask_2_price', 'ask_2_volume', 'ask_3_price', 
    'ask_3_volume'
]

modified_columns = [
    'ceiling', 'floor', 'ref_price',
    'prior_close_price', 'match_price', 
    'match_vol', 'accumulated_volume', 'accumulated_value', 
    'avg_match_price', 'highest', 'lowest',
    'foreign_sell_volume', 'foreign_buy_volume', 'bid_1_price', 'bid_1_volume', 'bid_2_price', 
    'bid_2_volume', 'bid_3_price', 'bid_3_volume', 'ask_1_price', 
    'ask_1_volume', 'ask_2_price', 'ask_2_volume', 'ask_3_price', 
    'ask_3_volume'
]

# Lọc DataFrame để chỉ giữ lại các cột cần thiết
price_data = price_data[required_columns]
price_data = price_data.loc[:, ~price_data.columns.duplicated()]
price_data[modified_columns] = price_data[modified_columns].fillna(0)
for column in modified_columns:
    price_data[column] = price_data[column]/1000
    
# In ra dữ liệu
print(price_data.to_string())

# Đặt tên file Excel
# file_name = 'price_data.xlsx'

# # Lưu DataFrame vào file Excel
# price_data.to_excel(file_name, index=False)
# print(f'Data đã được lưu vào file {file_name}')

# # Chèn dữ liệu vào bảng PostgreSQL
# postgre.insert_dataframe_to_table(price_data, table_name)

       symbol  ceiling   floor  ref_price  stock_type exchange last_trading_date  listed_share        type          id                                                                     organ_name  prior_close_price  match_price  match_vol  accumulated_volume  accumulated_value  avg_match_price  highest  lowest match_type  foreign_sell_volume  foreign_buy_volume  current_room    total_room  bid_1_price  bid_1_volume  bid_2_price  bid_2_volume  bid_3_price  bid_3_volume  ask_1_price  ask_1_volume  ask_2_price  ask_2_volume  ask_3_price  ask_3_volume
0         AAV     8.30    6.90       7.60       STOCK      HNX               NaN      68987661       STOCK         NaN                                                      Công ty Cổ phần AAV Group               0.00         7.50        1.0              1342.8           10.10747         7.527160     7.70    7.40          s                0.000              18.100      33719009           NaN         7.50          14.0         7.40         29

In [24]:
df = stock.finance.ratio(symbol ='A32',period='year', lang='vi', dropna=True)
# print(df.info())
df.columns = ['_'.join(col).strip() for col in df.columns.values]
df.reset_index(inplace=True)
required_columns = [
                    'Meta_CP',  
                    'Meta_Năm',  
                    'Meta_Kỳ',  
                    'Chỉ tiêu cơ cấu nguồn vốn_(Vay NH+DH)/VCSH',  
                    'Chỉ tiêu cơ cấu nguồn vốn_Nợ/VCSH',  
                    'Chỉ tiêu cơ cấu nguồn vốn_TSCĐ / Vốn CSH',  
                    'Chỉ tiêu cơ cấu nguồn vốn_Vốn CSH/Vốn điều lệ',  
                    'Chỉ tiêu hiệu quả hoạt động_Vòng quay tài sản',  
                    'Chỉ tiêu hiệu quả hoạt động_Vòng quay TSCĐ',  
                    'Chỉ tiêu hiệu quả hoạt động_Số ngày thu tiền bình quân',  
                    'Chỉ tiêu hiệu quả hoạt động_Số ngày tồn kho bình quân',  
                    'Chỉ tiêu hiệu quả hoạt động_Số ngày thanh toán bình quân',  
                    'Chỉ tiêu hiệu quả hoạt động_Chu kỳ tiền',  
                    'Chỉ tiêu hiệu quả hoạt động_Vòng quay hàng tồn kho',  
                    'Chỉ tiêu khả năng sinh lợi_Biên EBIT (%)',  
                    'Chỉ tiêu khả năng sinh lợi_Biên lợi nhuận gộp (%)',  
                    'Chỉ tiêu khả năng sinh lợi_Biên lợi nhuận ròng (%)',  
                    'Chỉ tiêu khả năng sinh lợi_ROE (%)',  
                    'Chỉ tiêu khả năng sinh lợi_ROIC (%)',  
                    'Chỉ tiêu khả năng sinh lợi_ROA (%)',  
                    'Chỉ tiêu khả năng sinh lợi_EBITDA (Tỷ đồng)',  
                    'Chỉ tiêu khả năng sinh lợi_EBIT (Tỷ đồng)',  
                    'Chỉ tiêu khả năng sinh lợi_Tỷ suất cổ tức (%)',  
                    'Chỉ tiêu thanh khoản_Chỉ số thanh toán hiện thời',  
                    'Chỉ tiêu thanh khoản_Chỉ số thanh toán tiền mặt',  
                    'Chỉ tiêu thanh khoản_Chỉ số thanh toán nhanh',  
                    'Chỉ tiêu thanh khoản_Khả năng chi trả lãi vay',  
                    'Chỉ tiêu thanh khoản_Đòn bẩy tài chính',  
                    'Chỉ tiêu định giá_Vốn hóa (Tỷ đồng)',  
                    'Chỉ tiêu định giá_Số CP lưu hành (Triệu CP)',  
                    'Chỉ tiêu định giá_P/E',  
                    'Chỉ tiêu định giá_P/B',  
                    'Chỉ tiêu định giá_P/S',  
                    'Chỉ tiêu định giá_P/Cash Flow',  
                    'Chỉ tiêu định giá_EPS (VND)',  
                    'Chỉ tiêu định giá_BVPS (VND)',  
                    'Chỉ tiêu định giá_EV/EBITDA'  
                ]

                # Thêm các cột thiếu với giá trị mặc định là 0
 
for col in required_columns:
    if col not in df.columns:
        df[col] = 0.0  
df = df[required_columns]
print(df.columns)


Index(['Meta_CP', 'Meta_Năm', 'Meta_Kỳ',
       'Chỉ tiêu cơ cấu nguồn vốn_(Vay NH+DH)/VCSH',
       'Chỉ tiêu cơ cấu nguồn vốn_Nợ/VCSH',
       'Chỉ tiêu cơ cấu nguồn vốn_TSCĐ / Vốn CSH',
       'Chỉ tiêu cơ cấu nguồn vốn_Vốn CSH/Vốn điều lệ',
       'Chỉ tiêu hiệu quả hoạt động_Vòng quay tài sản',
       'Chỉ tiêu hiệu quả hoạt động_Vòng quay TSCĐ',
       'Chỉ tiêu hiệu quả hoạt động_Số ngày thu tiền bình quân',
       'Chỉ tiêu hiệu quả hoạt động_Số ngày tồn kho bình quân',
       'Chỉ tiêu hiệu quả hoạt động_Số ngày thanh toán bình quân',
       'Chỉ tiêu hiệu quả hoạt động_Chu kỳ tiền',
       'Chỉ tiêu hiệu quả hoạt động_Vòng quay hàng tồn kho',
       'Chỉ tiêu khả năng sinh lợi_Biên EBIT (%)',
       'Chỉ tiêu khả năng sinh lợi_Biên lợi nhuận gộp (%)',
       'Chỉ tiêu khả năng sinh lợi_Biên lợi nhuận ròng (%)',
       'Chỉ tiêu khả năng sinh lợi_ROE (%)',
       'Chỉ tiêu khả năng sinh lợi_ROIC (%)',
       'Chỉ tiêu khả năng sinh lợi_ROA (%)',
       'Chỉ tiêu khả năng sinh 

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


In [2]:
print(stock.quote.intraday(symbol = 'AAA'))

                  time   price  volume match_type         id
99 2024-12-05 14:18:02  8800.0    6000       Sell  257495785
96 2024-12-05 14:18:02  8800.0     100       Sell  257495788
98 2024-12-05 14:18:02  8800.0     800       Sell  257495786
97 2024-12-05 14:18:02  8800.0    4500       Sell  257495787
95 2024-12-05 14:18:08  8800.0    1000        Buy  257495979
..                 ...     ...     ...        ...        ...
4  2024-12-05 14:29:17  8820.0     100        Buy  257530955
3  2024-12-05 14:29:17  8820.0     600        Buy  257530956
2  2024-12-05 14:29:56  8820.0     400        Buy  257533108
1  2024-12-05 14:29:56  8820.0    1600        Buy  257533109
0  2024-12-05 14:45:01  8810.0  135500    ATO/ATC  257534230

[100 rows x 5 columns]
